<a href="https://colab.research.google.com/github/donaldocelaj/DS-Unit-2-Sprint-4-Model-Validation/blob/master/Sprint%204%20Unit%202%20completion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Data Science Unit 2 Sprint Challenge 4 — Model Validation

Follow the instructions for each numbered part to earn a score of 2. See the bottom of the notebook for a list of ways you can earn a score of 3.

## Predicting Blood Donations

Our dataset is from a mobile blood donation vehicle in Taiwan. The Blood Transfusion Service Center drives to different universities and collects blood as part of a blood drive.

The goal is to predict the last column, whether the donor made a donation in March 2007, using information about each donor's history. We'll measure success using recall score as the model evaluation metric.

Good data-driven systems for tracking and predicting donations and supply needs can improve the entire supply chain, making sure that more patients get the blood transfusions they need.

#### Run this cell to load the data:

In [24]:
!pip install category_encoders

    100% |████████████████████████████████| 61kB 2.4MB/s 


In [0]:
import warnings
import category_encoders as ce
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.model_selection import train_test_split

warnings.filterwarnings(action='ignore', category=DataConversionWarning)

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data')

df = df.rename(columns={
    'Recency (months)': 'months_since_last_donation', 
    'Frequency (times)': 'number_of_donations', 
    'Monetary (c.c. blood)': 'total_volume_donated', 
    'Time (months)': 'months_since_first_donation', 
    'whether he/she donated blood in March 2007': 'made_donation_in_march_2007'
})

In [2]:
df.head(10)

,months_since_last_donation,number_of_donations,total_volume_donated,months_since_first_donation,made_donation_in_march_2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0
5,4,4,1000,4,0
6,2,7,1750,14,1
7,1,12,3000,35,0
8,2,9,2250,22,1
9,5,46,11500,98,1


In [26]:
df.isnull().sum()

months_since_last_donation     0
number_of_donations            0
total_volume_donated           0
months_since_first_donation    0
made_donation_in_march_2007    0
dtype: int64

## Part 1.1 — Begin with baselines

What **accuracy score** would you get here with a **"majority class baseline"?** 
 
(You don't need to split the data into train and test sets yet. You can answer this question either with a scikit-learn function or with a pandas function.)

In [27]:
df['made_donation_in_march_2007'].value_counts(normalize=True)

0    0.762032
1    0.237968
Name: made_donation_in_march_2007, dtype: float64

In [0]:
majority = 0
prediction = [majority] * len(df['made_donation_in_march_2007'])

In [0]:
from sklearn.metrics import accuracy_score

In [13]:
accuracy_score(df['made_donation_in_march_2007'], prediction)

0.7620320855614974

What **recall score** would you get here with a **majority class baseline?**

(You can answer this question either with a scikit-learn function or with no code, just your understanding of recall.)

In [0]:
from sklearn.metrics import recall_score

In [22]:
recall = recall_score(df['made_donation_in_march_2007'], prediction, average='micro')
print('Recall Score: ', recall)

Recall Score:  0.7620320855614974


## Part 1.2 — Split data

In this Sprint Challenge, you will use "Cross-Validation with Independent Test Set" for your model evaluation protocol.

First, **split the data into `X_train, X_test, y_train, y_test`**, with random shuffle. (You can include 75% of the data in the train set, and hold out 25% for the test set.)


In [29]:
X = df.drop(columns='made_donation_in_march_2007')
y = df['made_donation_in_march_2007']
X.head(10)

,months_since_last_donation,number_of_donations,total_volume_donated,months_since_first_donation
0,2,50,12500,98
1,0,13,3250,28
2,1,16,4000,35
3,2,20,5000,45
4,1,24,6000,77
5,4,4,1000,4
6,2,7,1750,14
7,1,12,3000,35
8,2,9,2250,22
9,5,46,11500,98


In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.25)

In [259]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((561, 4), (187, 4), (561,), (187,))

## Part 2.1 — Make a pipeline

Make a **pipeline** which includes:
- Preprocessing with any scikit-learn [**Scaler**](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)
- Feature selection with **[`SelectKBest`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)([`f_classif`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html))**
- Classification with [**`LogisticRegression`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [0]:
from sklearn.feature_selection import f_classif

In [261]:
pipeline = make_pipeline(SelectKBest(f_classif, k=4),
                         StandardScaler(),
                         LogisticRegression(solver='lbfgs', max_iter=1000))

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('selectkbest', SelectKBest(k=4, score_func=<function f_classif at 0x7f0e6c2640d0>)), ('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('logisticregression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

## Part 2.2 — Do Grid Search Cross-Validation

Do [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) with your pipeline. Use **5 folds** and **recall score**.

Include these **parameters for your grid:**

#### `SelectKBest`
- `k : 1, 2, 3, 4`

#### `LogisticRegression`
- `class_weight : None, 'balanced'`
- `C : .0001, .001, .01, .1, 1.0, 10.0, 100.00, 1000.0, 10000.0`


**Fit** on the appropriate data.

In [0]:
sorted(metrics.SCORERS.keys())

In [0]:
from sklearn.svm import SVC

In [0]:
param_grid = {'logisticregression__C': [.0001, .001, .01, 1.0, 10.0, 100.00, 1000.0, 10000.0],              
              'selectkbest__k': [1, 2, 3, 4],
              'logisticregression__class_weight' : [None, 'balanced']
              }

In [0]:

gridsearch = GridSearchCV(pipeline, 
                          param_grid,
                          cv=5, 
                          scoring='recall',
                          verbose=1,
             )

In [0]:
gridsearch.fit(X_train, Y_train)

## Part 3 — Show best score and parameters

Display your **best cross-validation score**, and the **best parameters** (the values of `k, class_weight, C`) from the grid search.

(You're not evaluated here on how good your score is, or which parameters you find. You're only evaluated on being able to display the information. There are several ways you can get the information, and any way is acceptable.)

In [266]:
print('Best Validation Score: ', gridsearch.best_score_)

Best Validation Score:  0.7863697667619235


In [0]:
dir(gridsearch)

In [268]:
print('Best Parameters: ', gridsearch.best_params_)

Best Parameters:  {'logisticregression__C': 1.0, 'logisticregression__class_weight': 'balanced', 'selectkbest__k': 2}


In [0]:
gs_df = pd.DataFrame(gridsearch.cv_results_).sort_values(by='rank_test_score')

In [270]:
gs_df.head(5)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_logisticregression__C,param_logisticregression__class_weight,param_selectkbest__k,params,rank_test_score,split0_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
45,0.005759,0.001501,0.78637,0.788566,100,balanced,2,"{'logisticregression__C': 100.0, 'logisticregr...",1,0.857143,...,0.62963,0.825688,0.851852,0.770642,0.740741,0.807339,0.000150,0.000015,0.089674,0.023558
61,0.005475,0.001429,0.78637,0.788566,10000,balanced,2,"{'logisticregression__C': 10000.0, 'logisticre...",1,0.857143,...,0.62963,0.825688,0.851852,0.770642,0.740741,0.807339,0.000075,0.000019,0.089674,0.023558
29,0.005834,0.001501,0.78637,0.788566,1,balanced,2,"{'logisticregression__C': 1.0, 'logisticregres...",1,0.857143,...,0.62963,0.825688,0.851852,0.770642,0.740741,0.807339,0.000074,0.000034,0.089674,0.023558
30,0.005964,0.001510,0.78637,0.788566,1,balanced,3,"{'logisticregression__C': 1.0, 'logisticregres...",1,0.857143,...,0.62963,0.825688,0.851852,0.770642,0.740741,0.807339,0.000101,0.000027,0.089674,0.023558
54,0.006003,0.001501,0.78637,0.788566,1000,balanced,3,"{'logisticregression__C': 1000.0, 'logisticreg...",1,0.857143,...,0.62963,0.825688,0.851852,0.770642,0.740741,0.807339,0.000131,0.000024,0.089674,0.023558


## Part 4 — Calculate classification metrics from a confusion matrix

Suppose this is the confusion matrix for your binary classification model:

<table>
  <tr>
    <th colspan="2" rowspan="2"></th>
    <th colspan="2">Predicted</th>
  </tr>
  <tr>
    <th>Negative</th>
    <th>Positive</th>
  </tr>
  <tr>
    <th rowspan="2">Actual</th>
    <th>Negative</th>
    <td>85</td>
    <td>58</td>
  </tr>
  <tr>
    <th>Positive</th>
    <td>8</td>
    <td>36</td>
  </tr>
</table>

Calculate accuracy

In [271]:
accuracy = (36+85)/(85+58+8+36)
print('Accuracy: ', accuracy)

Accuracy:  0.6470588235294118


Calculate precision

In [272]:
precision = 36/(36+58)
print('Precision: ', precision)

Precision:  0.3829787234042553


Calculate recall

In [273]:
recall = 36/(36+8)
print('Recall: ', recall)

Recall:  0.8181818181818182


## BONUS — How you can earn a score of 3

### Part 1
Do feature engineering, to try improving your cross-validation score.

### Part 2
Add transformations in your pipeline and parameters in your grid, to try improving your cross-validation score.

### Part 3
Show names of selected features. Then do a final evaluation on the test set — what is the test score?

### Part 4
Calculate F1 score and False Positive Rate. 

In [274]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)
X_train_polynomial = poly.fit_transform(X_train)

print(X_train.shape, X_train_polynomial.shape)

(561, 4) (561, 15)


In [0]:
pipe = make_pipeline(SelectKBest(f_classif, k=4),
                     StandardScaler(),
                     LogisticRegression(solver='lbfgs', max_iter=1000))
param_grid = {
    'selectkbest__k' : [ 1, 2, 3, 4],
    'logisticregression__class_weight' : [None, 'balanced'],
    'logisticregression__C' : [.0001, .001, .01, .1, 1.0, 10.0, 100.00, 1000.0, 10000.0]
}

gridsearch = GridSearchCV(pipe, 
                  param_grid= param_grid, 
                  cv= 5, 
                  scoring= 'recall', 
                  verbose=1)

gridsearch.fit(X_train_polynomial, Y_train)

In [0]:

validation_score = gridsearch.best_score_

In [277]:
print(validation_score)

0.7789755632892886
